# Train ALL 4 Classification Models (Optimized)
**Author:** G8  
**Task:** 2.1 - Train 4 CNN Models  
**Strategy:** Unfreeze top layers for ALL transfer learning models  

**Models:**
1. Custom CNN (simplified architecture)
2. ResNet50 (with unfrozen top 30 layers)
3. EfficientNet-B0 (with unfrozen top 20 layers)
4. MobileNetV2 (with unfrozen top 20 layers)

In [11]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import Adam  # For M1

np.random.seed(42)
tf.random.set_seed(42)

print("Setup complete")

print("Libraries imported!")
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

Setup complete
Libraries imported!
TensorFlow: 2.14.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
# Config
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "single_objects"
MODELS_PATH = PROJECT_ROOT / "models" / "classification"
LOGS_PATH = PROJECT_ROOT / "logs" / "classification"

MODELS_PATH.mkdir(parents=True, exist_ok=True)
LOGS_PATH.mkdir(parents=True, exist_ok=True)

with open(PROJECT_ROOT / "data" / "class_mapping.json", 'r') as f:
    class_info = json.load(f)
NUM_CLASSES = class_info['num_classes']

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20  # Start with 5 for testing

print(f"Classes: {NUM_CLASSES}")

Classes: 39


In [ ]:
# Data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,               # Increased for more face angle variety
    width_shift_range=0.25,          # Horizontal shift
    height_shift_range=0.25,         # Vertical shift
    shear_range=0.2,                 # Light shear for pose variation
    zoom_range=0.25,                 # Zoom in/out for distance changes
    horizontal_flip=True,            # Flip horizontally (faces are symmetric)
    brightness_range=[0.7, 1.35],    # Strong brightness variation for lighting
    channel_shift_range=25.0,        # Color channel shift for robustness
    fill_mode='nearest'
)

# Validation & Test: only rescale (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_gen = train_datagen.flow_from_directory(
    DATA_PATH / 'train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    DATA_PATH / 'val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"Train: {train_gen.samples}, Val: {val_gen.samples}")

Found 2871 images belonging to 39 classes.
Found 611 images belonging to 39 classes.
Train: 2871, Val: 611


## Model 1: Custom CNN (Simplified)

In [29]:
def build_custom_cnn():
    """Simplified custom CNN with better regularization"""
    model = models.Sequential([
        # Block 1
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # Classifier
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0005),  # Higher LR for from-scratch
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building Custom CNN...")
model_cnn = build_custom_cnn()
print(f"Params: {model_cnn.count_params():,}")

Building Custom CNN...
Params: 1,697,191


In [30]:
# Train Custom CNN
print("="*80)
print("TRAINING CUSTOM CNN")
print("="*80)

callbacks_cnn = [
    ModelCheckpoint(str(MODELS_PATH / 'custom_cnn_best.h5'), monitor='val_accuracy', 
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_cnn = model_cnn.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_cnn, verbose=1
)

model_cnn.save(MODELS_PATH / 'custom_cnn_last.h5')
print(f"\nBest: {max(history_cnn.history['val_accuracy']):.4f}")

TRAINING CUSTOM CNN
Epoch 1/20
90/90 [==============================] - ETA: 0s - loss: 3.5615 - accuracy: 0.1550
Epoch 1: val_accuracy improved from -inf to 0.02455, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/custom_cnn_best.h5
90/90 [==============================] - 111s 1s/step - loss: 3.5615 - accuracy: 0.1550 - val_loss: 4.7250 - val_accuracy: 0.0245 - lr: 5.0000e-04
Epoch 2/20
90/90 [==============================] - ETA: 0s - loss: 2.7743 - accuracy: 0.2842
Epoch 2: val_accuracy did not improve from 0.02455
90/90 [==============================] - 103s 1s/step - loss: 2.7743 - accuracy: 0.2842 - val_loss: 4.9730 - val_accuracy: 0.0229 - lr: 5.0000e-04
Epoch 3/20
90/90 [==============================] - ETA: 0s - loss: 2.4082 - accuracy: 0.3769
Epoch 3: val_accuracy improved from 0.02455 to 0.03273, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE76

## Model 2: ResNet50 (Fine-tuned)

In [16]:
def build_resnet50():
    """ResNet50 with top layers unfrozen for fine-tuning"""
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 30 layers
    base.trainable = True
    for layer in base.layers[:-30]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0005),  # Higher for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building ResNet50...")
model_resnet = build_resnet50()
trainable = sum([tf.size(w).numpy() for w in model_resnet.trainable_weights])
print(f"Trainable params: {trainable:,}")

Building ResNet50...
Trainable params: 15,523,367


In [31]:
# Train ResNet50
print("="*80)
print("TRAINING RESNET50")
print("="*80)

callbacks_resnet = [
    ModelCheckpoint(str(MODELS_PATH / 'resnet50_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_resnet = model_resnet.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_resnet, verbose=1
)

model_resnet.save(MODELS_PATH / 'resnet50_last.h5')
print(f"\nBest: {max(history_resnet.history['val_accuracy']):.4f}")

TRAINING RESNET50
Epoch 1/20
90/90 [==============================] - ETA: 0s - loss: 4.0475 - accuracy: 0.0916
Epoch 1: val_accuracy improved from -inf to 0.14730, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/resnet50_best.h5
90/90 [==============================] - 46s 494ms/step - loss: 4.0475 - accuracy: 0.0916 - val_loss: 3.4146 - val_accuracy: 0.1473 - lr: 1.0000e-04
Epoch 2/20
90/90 [==============================] - ETA: 0s - loss: 3.9165 - accuracy: 0.0975
Epoch 2: val_accuracy improved from 0.14730 to 0.18985, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/resnet50_best.h5
90/90 [==============================] - 44s 484ms/step - loss: 3.9165 - accuracy: 0.0975 - val_loss: 3.3558 - val_accuracy: 0.1899 - lr: 1.0000e-04
Epoch 3/20
90/90 [==============================] - ETA: 0s

## Model 3: EfficientNet-B0 (Fine-tuned)

In [33]:
def build_efficientnet():
    """EfficientNet with top layers unfrozen"""
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 20 layers
    base.trainable = True
    for layer in base.layers[:-20]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building EfficientNet...")
model_eff = build_efficientnet()
trainable = sum([tf.size(w).numpy() for w in model_eff.trainable_weights])
print(f"Trainable params: {trainable:,}")

Building EfficientNet...
Trainable params: 2,026,839


In [34]:
# Train EfficientNet
print("="*80)
print("TRAINING EFFICIENTNET-B0")
print("="*80)

callbacks_eff = [
    ModelCheckpoint(str(MODELS_PATH / 'efficientnet_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_eff = model_eff.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_eff, verbose=1
)

model_eff.save(MODELS_PATH / 'efficientnet_last.h5')
print(f"\nBest: {max(history_eff.history['val_accuracy']):.4f}")

TRAINING EFFICIENTNET-B0
Epoch 1/20
90/90 [==============================] - ETA: 0s - loss: 3.7240 - accuracy: 0.0310
Epoch 1: val_accuracy improved from -inf to 0.03437, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/efficientnet_best.h5
90/90 [==============================] - 60s 449ms/step - loss: 3.7240 - accuracy: 0.0310 - val_loss: 3.7356 - val_accuracy: 0.0344 - lr: 3.0000e-04
Epoch 2/20
90/90 [==============================] - ETA: 0s - loss: 3.7021 - accuracy: 0.0268
Epoch 2: val_accuracy did not improve from 0.03437
90/90 [==============================] - 37s 413ms/step - loss: 3.7021 - accuracy: 0.0268 - val_loss: 3.7605 - val_accuracy: 0.0245 - lr: 3.0000e-04
Epoch 3/20
90/90 [==============================] - ETA: 0s - loss: 3.6470 - accuracy: 0.0414
Epoch 3: val_accuracy did not improve from 0.03437
90/90 [==============================] - 41s 454ms/step - loss: 3.6470 -

## Model 4: MobileNetV2 (Fine-tuned)

In [35]:
def build_mobilenet():
    """MobileNetV2 with top layers unfrozen"""
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 20 layers
    base.trainable = True
    for layer in base.layers[:-20]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building MobileNetV2...")
model_mobile = build_mobilenet()
trainable = sum([tf.size(w).numpy() for w in model_mobile.trainable_weights])
print(f"Trainable params: {trainable:,}")

Building MobileNetV2...
Trainable params: 1,881,959


In [36]:
# Train MobileNetV2
print("="*80)
print("TRAINING MOBILENETV2")
print("="*80)

callbacks_mobile = [
    ModelCheckpoint(str(MODELS_PATH / 'mobilenet_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_mobile = model_mobile.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_mobile, verbose=1
)

model_mobile.save(MODELS_PATH / 'mobilenet_last.h5')
print(f"\nBest: {max(history_mobile.history['val_accuracy']):.4f}")

TRAINING MOBILENETV2
Epoch 1/20
90/90 [==============================] - ETA: 0s - loss: 1.2771 - accuracy: 0.6897
Epoch 1: val_accuracy improved from -inf to 0.89198, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/mobilenet_best.h5
90/90 [==============================] - 26s 269ms/step - loss: 1.2771 - accuracy: 0.6897 - val_loss: 0.3668 - val_accuracy: 0.8920 - lr: 3.0000e-04
Epoch 2/20
90/90 [==============================] - ETA: 0s - loss: 0.1678 - accuracy: 0.9537
Epoch 2: val_accuracy improved from 0.89198 to 0.94108, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/CNN_Attendance_System/models/classification/mobilenet_best.h5
90/90 [==============================] - 25s 280ms/step - loss: 0.1678 - accuracy: 0.9537 - val_loss: 0.2141 - val_accuracy: 0.9411 - lr: 3.0000e-04
Epoch 3/20
90/90 [==============================] - ET

## Summary After small Epochs Test

In [37]:
results = [
    {'model': 'Custom CNN', 'best_val_acc': max(history_cnn.history['val_accuracy'])},
    {'model': 'ResNet50', 'best_val_acc': max(history_resnet.history['val_accuracy'])},
    {'model': 'EfficientNet', 'best_val_acc': max(history_eff.history['val_accuracy'])},
    {'model': 'MobileNetV2', 'best_val_acc': max(history_mobile.history['val_accuracy'])},
]

df = pd.DataFrame(results).sort_values('best_val_acc', ascending=False)
print("\n" + "="*80)
print("5-EPOCH TEST RESULTS")
print("="*80)
print(df.to_string(index=False))

print("\n" + "="*80)
print("DECISION")
print("="*80)
print("\nIf all models >70% after 5 epochs:")
print("  -> Increase EPOCHS to 50 and retrain all")
print("\nIf some models <50% after 5 epochs:")
print("  -> Focus only on models with >70%")
print("  -> Adjust LR for struggling models")

best = df.iloc[0]
print(f"\nCurrent best: {best['model']} at {best['best_val_acc']:.1%}")


5-EPOCH TEST RESULTS
       model  best_val_acc
 MobileNetV2      0.990180
  Custom CNN      0.864157
    ResNet50      0.417349
EfficientNet      0.068740

DECISION

If all models >70% after 5 epochs:
  -> Increase EPOCHS to 50 and retrain all

If some models <50% after 5 epochs:
  -> Focus only on models with >70%
  -> Adjust LR for struggling models

Current best: MobileNetV2 at 99.0%


## After verifying small epochs work, change EPOCHS=50 and rerun